<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#2023年春季版本" data-toc-modified-id="2023年春季版本-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>2023年春季版本</a></span></li><li><span><a href="#2022年春季版本" data-toc-modified-id="2022年春季版本-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2022年春季版本</a></span></li></ul></div>

In [1]:
!pip install dask

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 793.4 kB/s eta 0:00:00:010:00:01


In [3]:
cd_file_path= r'H:\cdinfo.txt'
infected_file_path= r'.\infected2.txt'

## 2023年春季版本

In [8]:
#导入相关的包
import pandas as pd
import dask.dataframe as dd
from timeit import default_timer as timer
# 记录代码运行时间
start = timer()
# 使用大数据处理框架Dask分块读取txt文件
cdinfo = dd.read_csv(cd_file_path, blocksize=100e6, names=['base', 'time', 'inorout', 'telephone'])
# cdinfo = pd.read_csv('cdinfo.txt',names = ['base','time','inorout','telephone'])
# pandas不能正常读取大数据集文件，即使成功读取，也极容易在操作中内存不足而崩溃

infected = pd.read_csv(infected_file_path, names=['telephone'])
infected_list = list(set(infected["telephone"].tolist()))
cdinfo_infected = cdinfo.loc[cdinfo["telephone"].isin(infected_list)]

# 感染列表转为pandas dataframe(考虑到只有10条数据)
cdinfo_infected = cdinfo_infected.compute()
infected_base = list(set(cdinfo_infected["base"]))
infected_base.sort()

# 如果某人在感染基站待过，标记为redmark
redmark_df = cdinfo.loc[cdinfo["base"].isin(infected_base)].compute()
redmark = list(set(redmark_df["telephone"].tolist()))
redmark.sort()

# 写入redmark.txt
with open('redmark/redmarkA03.txt', 'w') as fp:
    for item in redmark:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

# superredmark部分
infected_base_time = cdinfo_infected.groupby(["base", "telephone"])["time"].apply(list).reset_index( name='infected_time').drop("telephone", axis=1)

# 对于每一个被污染的基站，通过字典记录其被污染的时间段
# 通过使用字典而不使用列表而避免暴力搜索，大幅度加快寻找时间
infected_base_time_dict = {}
for index, row in infected_base_time.iterrows():
    if row["base"] not in infected_base_time_dict.keys():
        infected_base_time_dict[row["base"]] = [row["infected_time"]]
    else:
        infected_base_time_dict[row["base"]].append(row["infected_time"])

redmark_df = redmark_df.groupby(["base", "telephone"])["time"].apply(list).reset_index(name='stay_time')

# 以下代码在正式数据中临时加入
# 以避免某人去了某一基站两次
stay_time = redmark_df['stay_time'].tolist()
fmax = 0
for i in range(len(stay_time)):
    stay_time[i].sort()
    if len(stay_time[i]) >= fmax:
        fmax = len(stay_time[i])
print("fmax", fmax)
# fmax=4,意味着存在某人进出进出某相同基站

# 将dataframe拆分
split_redmark_df = pd.DataFrame(stay_time, columns=['start', 'end', "start1", "end1"])
# split_redmark_df = pd.DataFrame(redmark_df['stay_time'].tolist(),columns=['start','end'])

# 连接两个dataframe
redmark_df = pd.concat([redmark_df, split_redmark_df], axis=1).drop("stay_time", axis=1)

is_infected = []
# 判断被标记为redmark的人是否能够被标记为superredmark
# 通过从redmark中继续筛选，而不是从cdinfo.txt中重新查找，代码运行时间将被大幅缩短
for index, row in redmark_df.iterrows():
    flag = False
    for i in infected_base_time_dict[row["base"]]:
        if i[1] >= row["start"] and i[0] <= row["end"]:
            flag = True
        if i[1] >= row["start1"] and i[0] <= row["end1"]:
            flag = True
    is_infected.append(flag)

superredmark_df = redmark_df.loc[is_infected]
final = list(set(superredmark_df["telephone"].tolist()))
final.sort()

# 写入superredmark.txt
with open('superredmark/superredmarkA03.txt', 'w') as fp:
    for item in final:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')
print("共运行时间" + f"{timer() - start}")


Done
fmax 4
Done
共运行时间345.41821470000014


## 2022年春季版本

In [ ]:
import pandas as pd
import dask.dataframe as dd
from timeit import default_timer as timer

start = timer()
cdinfo = dd.read_csv('cdinfo.txt', blocksize=100e6, names=['base', 'time', 'inorout', 'telephone'])
# cdinfo = pd.read_csv('cdinfo.txt',names = ['base','time','inorout','telephone'])
# pandas will not read successfully, even if it can be read, it is likely to crash in subsequent processing.

infected = pd.read_csv('infected/infected03.txt', names=['telephone'])
infected_list = list(set(infected["telephone"].tolist()))
cdinfo_infected = cdinfo.loc[cdinfo["telephone"].isin(infected_list)]
# print(cdinfo_infected)

# convert to pandas dataframe(because it is small dataset)
cdinfo_infected = cdinfo_infected.compute()
infected_base = list(set(cdinfo_infected["base"]))
infected_base.sort()

# If somebody has stayed in a polluted base station, it is judged as a red code
redmark_df = cdinfo.loc[cdinfo["base"].isin(infected_base)].compute()
redmark = list(set(redmark_df["telephone"].tolist()))
redmark.sort()

# write to redmark.txt
with open('redmark/redmark03.txt', 'w') as fp:
    for item in redmark:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

# superredmark:
infected_base_time = cdinfo_infected.groupby(["base", "telephone"])["time"].apply(list).reset_index(
    name='infected_time').drop("telephone", 1)

# For each polluted base station, record the corresponding polluted time period through a dictionary
# The reason for using a dictionary is to quickly find the value by key and avoid brute force search
infected_base_time_dict = {}
for index, row in infected_base_time.iterrows():
    if row["base"] not in infected_base_time_dict.keys():
        infected_base_time_dict[row["base"]] = [row["infected_time"]]
    else:
        infected_base_time_dict[row["base"]].append(row["infected_time"])

redmark_df = redmark_df.groupby(["base", "telephone"])["time"].apply(list).reset_index(name='stay_time')

# The following code is temporarily added in the formal experiment to solve the problem
# that the same person has been to the same base station twice
stay_time = redmark_df['stay_time'].tolist()
fmax = 0
for i in range(len(stay_time)):
    stay_time[i].sort()
    if len(stay_time[i]) >= fmax:
        fmax = len(stay_time[i])
print("fmax", fmax)
# fmax=4, which means there is a person entering and leaving the same base station twice

# new df from the column of lists
split_redmark_df = pd.DataFrame(stay_time, columns=['start', 'end', "start1", "end1"])
# split_redmark_df = pd.DataFrame(redmark_df['stay_time'].tolist(),columns=['start','end'])

# connect two dataframes
redmark_df = pd.concat([redmark_df, split_redmark_df], axis=1).drop("stay_time", 1)
# print(redmark_df)
is_infected = []
# Determine whether each person marked with a red code can be marked as a superredmark person line by line
for index, row in redmark_df.iterrows():
    flag = False
    for i in infected_base_time_dict[row["base"]]:
        if i[1] >= row["start"] and i[0] <= row["end"]:
            flag = True
        if i[1] >= row["start1"] and i[0] <= row["end1"]:
            flag = True
    is_infected.append(flag)

# redmark_df["stay_time"][0]<=infected_base_time_dict[redmark_df["base"]][1] and redmark_df["stay_time"][
# 1]>=infected_base_time_dict[redmark_df["base"]][0]
superredmark_df = redmark_df.loc[is_infected]
final = list(set(superredmark_df["telephone"].tolist()))
final.sort()

# write to superredmark.txt
with open('superredmark/superredmark03.txt', 'w') as fp:
    for item in final:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')
print("共运行时间" + f"{timer() - start}")
